In [7]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from queue import Queue
import random

In [2]:
def create_dag(max_level=10, max_children=10, max_number_nodes=10, having_childs_prob=2):
    q = Queue()
    last_node = 0
    G = nx.Graph()
    G.add_node(0)
    q.put((0, 0))

    weights = [1/n for n in range(1, max_children+1)]

    while not q.empty():
        node, level = q.get()

        if not level == max_level and last_node <= max_number_nodes:
            has_childs = random.randint(1, having_childs_prob)

            if node == 0 or has_childs != 1:
                
                n_childs = random.choices(range(1, max_children + 1))[0]

                for c in range(n_childs):
                    # Nodes start from 0
                    if last_node == max_number_nodes - 1:
                        break

                    last_node += 1
                    G.add_node(last_node)
                    G.add_edge(node, last_node)
                    q.put((last_node, level + 1))

    return G

In [3]:
def create_dag_levelist(max_level=10, max_level_nodes=10 , max_number_nodes=10, having_childs_prob=2):
    q = Queue()
    last_node = 0
    last_level = 0
    nodes = [[0]]

    G = nx.DiGraph()
    G.add_node(0)

    q.put((last_node, last_level))

    while not q.empty():
        node, level = q.get()

        # Adding nodes to the new level
        if last_level == level and last_level < max_level:
            last_level += 1
            n_new_level_nodes = random.randint(1, max_level_nodes)
            new_level_nodes = []

            for i in range(0, n_new_level_nodes - 1 ):
                if last_node >= max_number_nodes:
                    break

                last_node += 1
                G.add_node(last_node)
                q.put((last_node, last_level))
                new_level_nodes.append(last_node)
            
            nodes.append(new_level_nodes)

        # Add edges if the current node has node incoming edges
        if node != 0 and G.in_degree(node) == 0:
            previous_level_nodes = nodes[level-1]
            n_parents = random.randint(1, previous_level_nodes.__len__())
            parent = random.sample(previous_level_nodes, n_parents)

            for p in parent:
                G.add_edge(p, node)


        # Adding edges for the current node
        if level < last_level:
            next_level_nodes = nodes[level+1]
            has_child = random.randint(1, having_childs_prob)
            if node == 0 and (next_level_nodes.__len__() > 0) or (has_child != 1 and next_level_nodes.__len__() > 0):
                # weights = [1/n for n in range(1, next_level_nodes.__le__())]
                # n_childs = random.choices(1, next_level_nodes.__len__(), weights=weights)[0]
                n_childs = random.randint(1, next_level_nodes.__len__())
                children = random.sample(next_level_nodes, n_childs)

                for child in children:
                    G.add_edge(node, child)

    return G


In [4]:
def adj_matrix(g:nx.Graph):
    n_nodes = g.number_of_nodes()
    adj = np.full((n_nodes, n_nodes), -1)
    nodes = g.nodes
    
    for node in nodes:
        edges = g.edges(node)
        for u, v  in edges:
            adj[u, v] = 0

    return adj


In [17]:
# g = create_dag(max_level=5, max_children=5, max_number_nodes=16, having_childs_prob=4)
g = create_dag_levelist(max_level=5, max_level_nodes=7, max_number_nodes=30, having_childs_prob=3)
g.number_of_nodes(), g.number_of_edges()

(13, 17)

In [18]:
g.edges

OutEdgeView([(0, 1), (0, 3), (0, 2), (1, 4), (1, 5), (1, 8), (1, 6), (1, 7), (3, 6), (3, 4), (3, 5), (4, 12), (4, 9), (4, 11), (4, 10), (6, 12), (6, 10)])

In [11]:
adj_matrix(g)

array([[-1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1

In [65]:
# def set_random_weigts(total_weight, adj:list):
#     adj = np.array(adj)
#     num_nodes = adj.shape[0]
    
#     for i in range(num_nodes):
        
#         if i == 0:
#             incoming_weight = total_weight
#         else:
#             incoming_weight = 0
#             for k in range(num_nodes):
#                 if adj[k, i] != -1:
#                     print(k, i, adj[k, i])
#                     incoming_weight += adj[k, i]
#             # incoming_weight = np.sum([adj[k, i] for k in range(num_nodes) if adj[k, i] != -1 ])    
#         print(incoming_weight)
            
#         if incoming_weight > 0:
#             remaining_weight = incoming_weight
            
#             children = [j for j in range(num_nodes) if adj[i, j] != -1]
            
#             n_children = children.__len__()
            
#             for c in range(n_children):
#                 if c != n_children - 1: 
#                     weight = random.randint(1, remaining_weight - (n_children - c))
#                 else:
#                     weight = remaining_weight
#                 remaining_weight -= weight
                
#                 adj[i, c] = weight
#     return adj
                

In [66]:
adj = [
    [-1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],

]

In [67]:
set_random_weigts(1000, adj)

1000
0 1 896
896
0 2 0
0
1 3 0
0
1 4 0
2 4 0
0
4 5 0
0
4 6 0
0
4 7 0
0
5 8 0
7 8 0
0
5 9 0
6 9 0
0
9 10 0
0
8 11 0
9 11 0
0
8 12 0
9 12 0
0
8 13 0
9 13 0
0
9 14 0
0


array([[104, 896,   0,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1],
       [433, 463,  -1,   0,   0,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,   0,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,  -1,   0,   0,   0,  -1,  -1,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,   0,   0,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,   0,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,   0,  -1,  -1,  -1,  -1,
         -1,  -1],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,   0,   0,
          0,  -1],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,   0,   0,   0,
          0,   0],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
       

In [ ]:
adj = [
    [-1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],

]